In [1]:
from langchain_community.document_loaders import PyPDFLoader

# Load a PDF document
loader = PyPDFLoader("./sew_docs/sew_docs.pdf")
documents = loader.load()
documents


[Document(metadata={'producer': 'Acrobat Distiller 7.0 (Windows); modified using iTextSharp 4.1.6 by 1T3XT', 'creator': 'FrameMaker 7.2', 'creationdate': '2008-05-20T14:31:50+00:00', 'title': '11358912.book', 'author': 'DESCHSTE', 'moddate': '2014-08-01T17:05:21+02:00', 'source': './sew_docs/sew_docs.pdf', 'total_pages': 232, 'page': 0, 'page_label': '1'}, page_content='Gearmotors \\ Industrial Gear Units \\ Drive Electronics \\ Drive Automation \\ Services\nGear Units and Gearmotors \nManual\nEdition 06/2008\n11358912 / CA'),
 Document(metadata={'producer': 'Acrobat Distiller 7.0 (Windows); modified using iTextSharp 4.1.6 by 1T3XT', 'creator': 'FrameMaker 7.2', 'creationdate': '2008-05-20T14:31:50+00:00', 'title': '11358912.book', 'author': 'DESCHSTE', 'moddate': '2014-08-01T17:05:21+02:00', 'source': './sew_docs/sew_docs.pdf', 'total_pages': 232, 'page': 1, 'page_label': '2'}, page_content='SEW-EURODRIVE – Driving the world'),
 Document(metadata={'producer': 'Acrobat Distiller 7.0 (W

In [ ]:
import os
from dotenv import load_dotenv
load_dotenv()  #load all the environment variables


In [4]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

text_splitter=RecursiveCharacterTextSplitter(chunk_size=1000,chunk_overlap=10)
splitted_documents=text_splitter.split_documents(documents)


In [12]:
from langchain_community.embeddings import OllamaEmbeddings

embeddings = OllamaEmbeddings(model="mxbai-embed-large:latest")

In [10]:
# embedding_vector = embeddings.embed_documents(splitted_documents)

In [19]:
from langchain_community.vectorstores import Chroma
vectordb=Chroma.from_documents(documents= splitted_documents, embedding=embeddings)
vectordb

In [31]:
query = "How to do Corrosion and surface protection"
response = vectordb.similarity_search(query)
response


[Document(metadata={'page_label': '16', 'creationdate': '2008-05-20T14:31:50+00:00', 'author': 'DESCHSTE', 'page': 15, 'producer': 'Acrobat Distiller 7.0 (Windows); modified using iTextSharp 4.1.6 by 1T3XT', 'moddate': '2014-08-01T17:05:21+02:00', 'title': '11358912.book', 'total_pages': 232, 'source': './sew_docs/sew_docs.pdf', 'creator': 'FrameMaker 7.2'}, page_content='Stainless steel output shaft Surface protection through high-\nquality material\nParticularly exacting applications in \nterms of exterior surface protection'),
 Document(metadata={'title': '11358912.book', 'moddate': '2014-08-01T17:05:21+02:00', 'total_pages': 232, 'creator': 'FrameMaker 7.2', 'page_label': '15', 'page': 14, 'author': 'DESCHSTE', 'producer': 'Acrobat Distiller 7.0 (Windows); modified using iTextSharp 4.1.6 by 1T3XT', 'creationdate': '2008-05-20T14:31:50+00:00', 'source': './sew_docs/sew_docs.pdf'}, page_content='Manual – Gear Units and Gearmotors  15\n2 Corrosion and surface protection\nProduct Descr

## Next Step: Add an LLM to Turn Vector Results into Answers
Let’s now use an LLM via Ollama (like llama3.2:latest ) to answer questions based on the retrieved chunks.

You’ll do this by creating a RetrievalQA chain:

In [43]:
from langchain_community.llms import Ollama
from langchain.chains import RetrievalQA

# Load Ollama LLM
llm = Ollama(model="llama3.2:latest")  # or any other model you like


In [44]:

# Create a QA chain that uses your Chroma retriever
qa_chain = RetrievalQA.from_chain_type(
    llm=llm,
    retriever=vectordb.as_retriever(),
    return_source_documents=True  # optional
)


In [45]:

# Ask a natural language question
query = "How to do Corrosion and surface protection?"
result = qa_chain.invoke({"query":query})

# Print the final answer
print("Answer:", result["result"])

Answer: The question is: "How to do Corrosion and surface protection?" According to the provided context, there are several measures offered by SEW-EURODRIVE for corrosion and surface protection:

1. Corrosion protection KS for motors includes:
* Stainless steel retaining screws
* Stainless steel nameplates or equivalent corrosion-resistant material
* Interior motor components protected with a corrosion-resistant material
* Additional sealing for brake motors
* Condensation drains (AC motor)
2. Gearmotor output shafts can be protected with optional coatings when operating in severe environments.
3. NOCO fluid is supplied with every hollow shaft gear unit and can also be used to protect machined metal surfaces that do not have corrosion protection.

Additionally, the Kanisil coating surface coating of the contact surface of the oil seal is an option for severe environmental pollution.

It's worth noting that some options (like motors with a forced cooling fan or spreadshaft encoder) are

##  To maintain session history of each chat (just like a real chatbot), you need to Use ConversationBufferMemory in LangChain

In [46]:
from langchain.memory import ConversationBufferMemory

In [47]:
from langchain.chains import ConversationalRetrievalChain # Use ConversationalRetrievalChain instead of RetrievalQA:
 

In [48]:
# Create memory buffer to track history
memory = ConversationBufferMemory(memory_key="chat_history", return_messages=True)

# Build conversational chain
qa_chain = ConversationalRetrievalChain.from_llm(
    llm=llm,
    retriever=vectordb.as_retriever(),
    memory=memory,
    return_source_documents=False  # Set to True if you want to debug
)


In [ ]:
chat_history = []  # Just for optional external display

# while True:
#     question = input("\nYou: ")
#     if question.lower() in ['exit', 'quit']:
#         break

#     result = qa_chain.invoke({"question": question})
#     print("Bot:", result["answer"])

#     chat_history.append((question, result["answer"]))



Bot: I don't know the answer to that question. The context provided only discusses the products, services, and documentation offered by SEW-EURODRIVE, but it doesn't explicitly state what "SEW" stands for.
Bot: According to the provided context, SEW-EURODRIVE offers a wide range of products and systems for power transmission and motion control, including gear units and gearmotors. They provide solutions for various industries such as automotive, food and beverage processing, packaging, building materials, water and wastewater treatment, logistics and transportation, and entertainment.
Bot: According to the provided context, some of the specific products mentioned in the documentation offered by SEW-EURODRIVE include:

1. Helical gear units and helical gearmotors
2. Parallel shaft helical gear units and parallel shaft helical gearmotors
3. Helical-bevel gear units and helical-bevel gearmotors
4. Helical-worm gear units and helical-worm gearmotors
5. Spiroplan® gearmotors
6. MOVIMOT® gea